# Amostragem populacional

In [1]:
import random
from itertools import groupby

## Amostragem Estratificada:

In [2]:
# Dados de exemplo
dados = [(1, 'A'), (2, 'A'), (3, 'B'), (4, 'B'), (5, 'B'), (6, 'C'), (7, 'C')]
print(dados)

[(1, 'A'), (2, 'A'), (3, 'B'), (4, 'B'), (5, 'B'), (6, 'C'), (7, 'C')]


In [3]:
# Dividindo os dados em estratos
dados_estratificados = {key: [item[0] for item in group] for key, group in groupby(sorted(dados, key=lambda x: x[1]), key=lambda x: x[1])}
print(dados_estratificados)

{'A': [1, 2], 'B': [3, 4, 5], 'C': [6, 7]}


In [4]:
# Selecionando uma amostra de cada estrato
amostra_estratificada = {key: random.sample(valores, 1) for key, valores in dados_estratificados.items()}
print("Amostra Estratificada:", amostra_estratificada)

Amostra Estratificada: {'A': [2], 'B': [4], 'C': [6]}


In [5]:
import os
import findspark
findspark.init(os.environ['SPARK_HOME'])

from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, desc

import scipy.stats as stats

# Inicializando a sessão do Spark
spark = SparkSession.builder.appName("ExemploAmostraDados").getOrCreate()

In [6]:
# Nome do arquivo CSV a ser lido
nome_arquivo = "..\\data\\caso_full.csv"

df = (
        spark
        .read
        .option("header", True)
        .option('encoding', 'UTF-8')
        .option('inferSchema', True)
        .csv(nome_arquivo)
    ).filter("place_type == 'city' AND new_confirmed > 0").select(['state', 'city', 'new_confirmed'])

df = df.withColumn("new_confirmed", col("new_confirmed").cast(IntegerType()))
df.limit(10).show(truncate=False)

+-----+----------+-------------+
|state|city      |new_confirmed|
+-----+----------+-------------+
|AC   |Rio Branco|3            |
|AC   |Rio Branco|1            |
|AC   |Rio Branco|3            |
|AC   |Rio Branco|4            |
|AC   |Rio Branco|6            |
|AC   |Rio Branco|4            |
|AC   |Rio Branco|2            |
|AC   |Rio Branco|2            |
|AC   |Acrelândia|2            |
|AC   |Porto Acre|1            |
+-----+----------+-------------+



In [7]:
states = df.select(['state', 'city']).distinct().collect()
lst_states = [(row.state, row.city) for row in states]
lst_states

[('BA', 'Itaberaba'),
 ('BA', 'Carinhanha'),
 ('BA', 'Igaporã'),
 ('BA', 'Rio de Contas'),
 ('ES', 'Guaçuí'),
 ('GO', 'Damianópolis'),
 ('AM', 'Novo Airão'),
 ('AM', 'Barcelos'),
 ('BA', 'Várzea da Roça'),
 ('CE', 'Iguatu'),
 ('AC', 'Rodrigues Alves'),
 ('CE', 'Ibaretama'),
 ('GO', 'Corumbá de Goiás'),
 ('GO', 'Varjão'),
 ('AC', 'Epitaciolândia'),
 ('AL', 'Palmeira dos Índios'),
 ('BA', 'Santa Luzia'),
 ('BA', 'Valença'),
 ('CE', 'Itaitinga'),
 ('CE', 'Paramoti'),
 ('GO', 'Nova Veneza'),
 ('GO', 'Quirinópolis'),
 ('AM', 'Rio Preto da Eva'),
 ('AM', 'Uarini'),
 ('AP', 'Pracuúba'),
 ('CE', 'Pires Ferreira'),
 ('GO', 'Abadia de Goiás'),
 ('AC', 'Jordão'),
 ('AL', 'Murici'),
 ('BA', 'Araci'),
 ('BA', 'Terra Nova'),
 ('BA', 'Santa Rita de Cássia'),
 ('GO', 'Santo Antônio de Goiás'),
 ('BA', 'Santa Bárbara'),
 ('BA', 'Riachão das Neves'),
 ('CE', 'Ubajara'),
 ('CE', 'Altaneira'),
 ('GO', 'Trindade'),
 ('GO', 'Aparecida do Rio Doce'),
 ('GO', 'Rio Quente'),
 ('GO', 'Davinópolis'),
 ('BA', 'Va

In [8]:
def calc_confiance(df_confiance):
    df_summary = df_confiance.select(['new_confirmed']).summary(['mean', 'stddev', 'max'])
    # Calculando as estatísticas
    media = df_summary.filter("summary = 'mean'").select('new_confirmed').first()[0]
    max = df_summary.filter("summary = 'max'").select('new_confirmed').first()[0]
    desvio_padrao = df_summary.filter("summary = 'stddev'").select('new_confirmed').first()[0]
    total_rows = df_confiance.count()

    # Exibindo as estatísticas
    print("Média dos new_confirmed:", media)
    print("Máximo dos new_confirmed:", max)
    print("Desvio padrão dos new_confirmed:", desvio_padrao)
    print("Totao de elementos da amostra:", total_rows)


    # Calculando scale a ser aplicada para o intervalo de confiança
    s = (float(max) - float(media)) / float(desvio_padrao)

    # Quantidade maxima de registro a ser analisado
    perc_dados = 0.95

    # Calculando intervalo de confiança para a média com 90% de confiança
    intervalo_confianca = stats.norm.interval(confidence=perc_dados, loc=float(media), scale=s)
    min_inter = intervalo_confianca[0]
    max_inter = intervalo_confianca[1]

    return min_inter, max_inter

In [9]:
count = df.select(['state', 'city']).groupby(['state', 'city']).count()
min_count = count.sort("count", ascending=True).select('count').first()[0]
min_count

1

In [10]:
for state in lst_states:
    try:
        dados_cat = df.filter(f"state == '{state[0]}' AND city == '{state[1]}'")
        
        if min_count < dados_cat.count():
            min, max = calc_confiance(dados_cat)
            dados_cat = dados_cat.filter(f"new_confirmed >= {min} AND new_confirmed <= {max}")
            print(f"state: {state[0]} AND city:{state[1]} count:{dados_cat.count()}")
            print("-------------------------------------------------------------------------------- \n")
            
            dados_cat = dados_cat.limit(min_count)
            
            (
                dados_cat
                .write
                .mode("append")
                .option("overwriteSchema", "true")
                .save("../data/amostras")
            )
    except:
        print(f"Erro state: {state[0]} AND city:{state[1]}")
        print("-------------------------------------------------------------------------------- \n")


Média dos new_confirmed: 18.004694835680752
Máximo dos new_confirmed: 127
Desvio padrão dos new_confirmed: 19.740313520981765
Totao de elementos da amostra: 426
state: BA AND city:Itaberaba count:197
-------------------------------------------------------------------------------- 

Média dos new_confirmed: 5.179577464788732
Máximo dos new_confirmed: 57
Desvio padrão dos new_confirmed: 5.957992389980566
Totao de elementos da amostra: 284
state: BA AND city:Carinhanha count:280
-------------------------------------------------------------------------------- 

Média dos new_confirmed: 3.682758620689655
Máximo dos new_confirmed: 41
Desvio padrão dos new_confirmed: 4.74151793597417
Totao de elementos da amostra: 145
state: BA AND city:Igaporã count:143
-------------------------------------------------------------------------------- 

Média dos new_confirmed: 3.1041666666666665
Máximo dos new_confirmed: 27
Desvio padrão dos new_confirmed: 3.1691477104419925
Totao de elementos da amostra: 240

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "E:\\Developement\\afirmanet\\drives\\spark-3.5.1-bin-hadoop3\python\lib\py4j-0.10.9.7-src.zip\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "E:\\Developement\\afirmanet\\drives\\spark-3.5.1-bin-hadoop3\python\lib\py4j-0.10.9.7-src.zip\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\aview\AppData\Local\Programs\Python\Python311\Lib\socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


Erro state: BA AND city:Cachoeira
-------------------------------------------------------------------------------- 

Média dos new_confirmed: 14.684782608695652
Máximo dos new_confirmed: 284
Desvio padrão dos new_confirmed: 21.018751984356214
Totao de elementos da amostra: 552
state: AL AND city:Marechal Deodoro count:508
-------------------------------------------------------------------------------- 

